In [ ]:
# First, we add all the packages we need to the system.
# This is what enables us to run all the code below.

import os
from dotenv import load_dotenv

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration, OrchestrationHandoffs, RoundRobinGroupChatManager
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.agents.runtime import InProcessRuntime

In [ ]:
# Here, we connect to the Azure AI Foundry, and get the necessary environment variables
# including the project endpoint, model deployment name, and tenant ID.

load_dotenv()
endpoint = os.getenv("ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
api_key = os.getenv("API_KEY")
api_version = os.getenv("API_VERSION")

In [ ]:
# This block of code connects to the MCP Server
async with MCPStdioPlugin(
    name="InventoryManagementMCPPlugin",
    description="Inventory Management MCP Plugin",
    command="python",
    args=["server.py"],
) as inventory_plugin:
    pass
 
chat_service = AzureChatCompletion(
    endpoint=endpoint,
    deployment_name=model_deployment,
    api_version=api_version,
    api_key=api_key
)

In [ ]:
def get_agents() -> tuple[list[Agent], OrchestrationHandoffs]:
    # Returns a list of agents

    inventory_agent = ChatCompletionAgent(
        name="InventoryAgent",
        description="An agent that is knowledgeable about the inventory.",
        instructions="Handle research and information about the inventory, including stock levels and item details.",
        service=chat_service,
        plugins=[inventory_plugin],
    )

    accounting_agent = ChatCompletionAgent(
        name="AccountingAgent",
        description="An agent that is knowledgeable about accounting tasks.",
        instructions="Receive reports of inventory status, and prepare a list of items to order, keeping in mind a budget of $1000.",
        service=chat_service,
        plugins=[inventory_plugin],
    )

    inventory_purchasing_agent = ChatCompletionAgent(
        name="InventoryPurchasingAgent",
        description="An agent that is able to purchase inventory items.",
        instructions="Handle purchasing tasks for inventory items, and will purchase items that the accounting agent has recommended. Use tools to place orders, and then confirm the order.",
        service=chat_service,
        plugins=[inventory_plugin],
    )

    return [inventory_agent, accounting_agent, inventory_purchasing_agent]

In [ ]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    if message.content:
        print(f"🤖{message.name}\n{message.content}")

In [ ]:
await inventory_plugin.connect()

# 1. Create a group chat orchestration with a round robin manager
agents = get_agents()
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    # max_rounds is odd, so that the writer gets the last round
    manager=RoundRobinGroupChatManager(max_rounds=5),
    agent_response_callback=agent_response_callback,
)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await group_chat_orchestration.invoke(
    task="""
    Use the InventoryAgent to research all the products in the Inventory that are low in stock, and prepare a report for the AccountingAgent.
    Ensure the AccountingAgent returns a a list of items to order, keeping in mind a budget of $1000.
    Send the list of items to order to the InventoryPurchasingAgent, and update the stock with what the AccountingAgent has suggested.
    Don't ask if the stock should be updated, just do it.
    When done, use the tools to retrieve the new product inventory units in stock from the InventoryAgent, indicating the before and after stock levels.""",
    runtime=runtime,
)

# 4. Wait for the results
value = await orchestration_result.get()
print(f"***** Result *****\n{value}")

# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()
await inventory_plugin.close() if inventory_plugin else None